## Versions:

### Ways to replicate an environment
Using Virtualenv or Conda we can recreate enviroments with specific versions of libraries and even Python interpreter

- `pip freeze` (requirements.txt)
- `conda list` (environment.yaml)
- Docker Container + requirements.txt/environment.yaml


In [36]:
import numpy
import keras
import tensorflow
import sklearn
import h5py
import yaml
import joblib

In [46]:
print('NumPy: %s'%numpy.__version__ )
print('TensorFlow: %s'%tensorflow.__version__)
print('Scikit-Learn:  %s'%sklearn.__version__)
print('Keras: %s'%keras.__version__)
print('HDF5: %s'%h5py.__version__)
print('PyYAML: %s'%yaml.__version__)
print('Joblib: %s'%joblib.__version__)

NumPy: 1.18.1
TensorFlow: 1.14.0
Scikit-Learn:  0.22.1
Keras: 2.3.1
HDF5: 2.10.0
PyYAML: 5.3.1
Joblib: 0.14.1


### Utilities

In [126]:
def create_model():
    """
    Create a Neural network and return the trained model with the used features to train
    
    Output: Features, Target and model
    """
    try:
        # Import tools
        from keras.models import Sequential
        from keras.layers import Dense
        from keras.models import model_from_yaml
        import numpy
        import os
        # fix random seed for reproducibility
        seed = 7
        numpy.random.seed(seed)
        # load pima indians dataset
        dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
        # split into input (X) and output (Y) variables
        X = dataset[:,0:8]
        Y = dataset[:,8]
        # create model
        model = Sequential()
        model.add(Dense(12, input_dim=8, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Fit the model
        model.fit(X, Y, epochs=150, batch_size=10, verbose=0)
        # evaluate the model
        scores = model.evaluate(X, Y, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        return X, Y, model
    except Exception as e:
        print(e)
        return None

    
def write_file(path, string_values):
    """
    Write a string 
    """
    with open(path, "w") as file:
        file.write(string_values)
    print('File saved to disk')


def read_file(path):
    with open(path, 'r') as file:
        return file.read()


# https://stackoverflow.com/a/43374773/2108769
def scan_hdf5(path, recursive=True, tab_step=2):
    def scan_node(g, tabs=0):
        print(' ' * tabs, g.name)
        for k, v in g.items():
            if isinstance(v, h5py.Dataset):
                print(' ' * tabs + ' ' * tab_step + ' -', v.name)
            elif isinstance(v, h5py.Group) and recursive:
                scan_node(v, tabs=tabs + tab_step)
    with h5py.File(path, 'r') as f:
        scan_node(f)

# Create Model

In [75]:
X, Y, model = create_model()


model

accuracy: 78.12%


# Saving full model in HDF5 (Weights + NN Architecture)

In [83]:
full_model = model
full_model.save('full-model.h5')
print("Saved model to disk")

Saved model to disk


In [88]:
del full_model

In [89]:
full_model

NameError: name 'full_model' is not defined

In [90]:
from keras.models import load_model


full_model = load_model('full-model.h5')

score = full_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 78.39%


In [121]:
# Scanning the file

scan_hdf5('full-model.h5') # Arch + Weights

 /
   /model_weights
     /model_weights/dense_7
       /model_weights/dense_7/dense_7
         - /model_weights/dense_7/dense_7/bias:0
         - /model_weights/dense_7/dense_7/kernel:0
     /model_weights/dense_8
       /model_weights/dense_8/dense_8
         - /model_weights/dense_8/dense_8/bias:0
         - /model_weights/dense_8/dense_8/kernel:0
     /model_weights/dense_9
       /model_weights/dense_9/dense_9
         - /model_weights/dense_9/dense_9/bias:0
         - /model_weights/dense_9/dense_9/kernel:0
   /optimizer_weights
     /optimizer_weights/Adam_2
       - /optimizer_weights/Adam_2/iterations:0
     /optimizer_weights/training_2
       /optimizer_weights/training_2/Adam
         - /optimizer_weights/training_2/Adam/m_0_1:0
         - /optimizer_weights/training_2/Adam/m_1_1:0
         - /optimizer_weights/training_2/Adam/m_2_1:0
         - /optimizer_weights/training_2/Adam/m_3_1:0
         - /optimizer_weights/training_2/Adam/m_4_1:0
         - /optimizer_weights/tra

In [122]:
!head -n 155 full-model.h5



                    ��������x      ��������        `              �       �                                     TREE   ����������������                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      HEAP    X       0       �              model_weights   optimizer_weights              (                                     �       �       H        keras_version                                          @        backend                          
   model_config                     

### Saving model's architecture to JSON and model's architecture in H5

#### Arch

In [109]:
arch_json = model.to_json()

In [110]:
type(arch_json)

<class 'str'>

In [117]:
arch_json

'{"class_name": "Sequential", "config": {"name": "sequential_3", "layers": [{"class_name": "Dense", "config": {"name": "dense_7", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_8", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "acti

In [118]:
write_file('arch.json', arch_json)

File saved to disk


In [119]:
!cat arch.json

{"class_name": "Sequential", "config": {"name": "sequential_3", "layers": [{"class_name": "Dense", "config": {"name": "dense_7", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_8", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activ

#### Weights

In [129]:
# serialize weights to HDF5
model.save_weights("model_weights.h5")
print("Saved weights model to disk")

Saved weights model to disk


In [143]:
!head -n 155 model_weights.h5



                    ���������=      ��������        `              �       �                                      TREE   ����������������                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      HEAP    X               �              dense_7 dense_8 dense_9        8                                                     �       �       P        layer_names                             dense_7dense_8dense_9    @        backend                          
   weight_names             

In [144]:
scan_hdf5('model_weights.h5')

 /
   /dense_7
     /dense_7/dense_7
       - /dense_7/dense_7/bias:0
       - /dense_7/dense_7/kernel:0
   /dense_8
     /dense_8/dense_8
       - /dense_8/dense_8/bias:0
       - /dense_8/dense_8/kernel:0
   /dense_9
     /dense_9/dense_9
       - /dense_9/dense_9/bias:0
       - /dense_9/dense_9/kernel:0


#### A few moments later...

In [130]:
from keras.models import model_from_json



model_fromjsonh5 = model_from_json(read_file('arch.json'))



In [132]:
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [133]:
model_fromjsonh5.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

accuracy: 64.58%


In [134]:
model_fromjsonh5.load_weights("model_weights.h5")

In [135]:
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

accuracy: 78.39%


In [136]:
model_fromjsonh5.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

accuracy: 78.39%


------

### Serializing model's architecture to YAML and model's architecture in HDF5


#### Arch

In [137]:
arch_yaml = model.to_yaml()


write_file('arch.yaml', arch_yaml)

File saved to disk


In [138]:
print(arch_yaml)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple
      - null
      - 8
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceScaling
        config:
          distribution: uniform
          mode: fan_avg
          scale: 1.0
          seed: null
      kernel_regularizer: null
      name: dense_7
      trainable: true
      units: 12
      use_bias: true
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceSc

In [139]:
!cat arch.yaml

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple
      - null
      - 8
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceScaling
        config:
          distribution: uniform
          mode: fan_avg
          scale: 1.0
          seed: null
      kernel_regularizer: null
      name: dense_7
      trainable: true
      units: 12
      use_bias: true
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceSc

In [141]:
# serialize weights to HDF5
model.save_weights("model_weights.h5")
print("Saved weights model to disk")

Saved weights model to disk


In [142]:
scan_hdf5('model_weights.h5')

 /
   /dense_7
     /dense_7/dense_7
       - /dense_7/dense_7/bias:0
       - /dense_7/dense_7/kernel:0
   /dense_8
     /dense_8/dense_8
       - /dense_8/dense_8/bias:0
       - /dense_8/dense_8/kernel:0
   /dense_9
     /dense_9/dense_9
       - /dense_9/dense_9/bias:0
       - /dense_9/dense_9/kernel:0


### Load YAML and H5 model

In [70]:
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")



Loaded model from disk


In [72]:
# Compile before predictions
score = loaded_model.evaluate(X, Y, verbose=0)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [76]:
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 78.39%


In [20]:
from joblib import dump, load


In [21]:
dump(model, 'model.joblib')

['model.joblib']

In [23]:
model_job_lib = load('model.joblib')

In [24]:
score = model_job_lib.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 78.39%
